<a href="https://colab.research.google.com/github/sbogde/pandamonium/blob/main/dmi_03_amazon_reviews_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time
import warnings

time.sleep(12)
warnings.filterwarnings('ignore')

In [92]:
## Define URLs for Scraping Amazon Reviews
urls = [
    'https://www.amazon.co.uk/dp/B0C2PRHCD3',
    'https://www.amazon.co.uk/dp/B0CYZH3D7Y',
    'https://www.amazon.co.uk/dp/B0D2D66R82',
    'https://www.amazon.co.uk/dp/B0DP1YVFQL',
    'https://www.amazon.co.uk/dp/B0DK1KK1FM'
]

In [ ]:
reviews_list = []
ratings_list = []

# Iterate over each movie URL
for url in urls:
    print(f"Scraping: {url}")
    headers = {'User-Agent': 'Mozilla/5.0'}  # Prevent getting blocked
    page = requests.get(url, headers=headers)

    print(f"page: {page}\n")
    # print(f"page.content,: {page.content}\n")

    # Parse the page content
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find all review containers
    review_containers = soup.find_all('li', class_='review aok-relative')
    print(f"review_containers: {review_containers}\n")

    # Extract review text and rating
    for container in review_containers:
        # Extract rating
        rating_element = container.find('span', class_='a-icon-alt')
        print(f"rating_element: {rating_element}\n")

        rating = rating_element.get_text(strip=True) if rating_element else "No Review"
        print(f"rating: {rating}\n")

        # Extract review text
        review_element = container.find('div', class_='a-expander-content reviewText review-text-content a-expander-partial-collapse-content')
        # print(f"review_element: {review_element}\n")

        review_text = review_element.get_text(strip=True) if review_element else "NA"
        # print(f"review_text: {review_text}\n")

        # Append to lists
        reviews_list.append(review_text)
        ratings_list.append(rating)

        # print(f"Review: {review_text[:100]}...")  # Print preview
        # print(f"Rating: {rating}\n")

    # Sleep to prevent being blocked
    time.sleep(20)



Scraping: https://www.amazon.co.uk/dp/B0C2PRHCD3
page: <Response [500]>

review_containers: []

Scraping: https://www.amazon.co.uk/dp/B0CYZH3D7Y
page: <Response [200]>

review_containers: [<li class="review aok-relative" data-hook="review" id="R25FVAR7A3OHOY"><span class="a-list-item"><div class="a-row a-spacing-none" id="R25FVAR7A3OHOY-review-card"><div class="a-section celwidget" id="customer_review-R25FVAR7A3OHOY"><script>
  var metricsName = "DesktopCredibleBadgesInTopReviewsDetailPageCount"
</script>
<script>
      if(window.ue) {
        ue.count(metricsName, 0)
      }
    </script>
<div class="a-row a-spacing-mini" data-hook="genome-widget"><a class="a-profile" data-a-size="small" href="/gp/profile/amzn1.account.AF34EWNV256KTFA7LWUCSMTALKDA"><div aria-hidden="true" class="a-profile-avatar-wrapper"><div class="a-profile-avatar"><img class="a-lazy-loaded" data-src="https://m.media-amazon.com/images/S/amazon-avatars-global/default.png" src="https://images-na.ssl-images-amazon.com/

In [94]:
print(reviews_list[1], "   ", ratings_list[1])

IndexError: list index out of range

In [ ]:
movie = pd.DataFrame({'Review': reviews_list, 'Rating': ratings_list})

# Save to CSV
movie.to_csv('Workshop3_IMDB_Dataset.csv', index=False)

# Display first few rows
print(movie.head())
print(movie.info())

In [ ]:
comments=["This is a good film","Interesting film","I do not like film", "Such a wonderful movie","very interesting film",
         "This film is about love story","Do not watch this film","I love this film", "I do not know, I feel it is an interesting film",
         "I like to watch film again", "This film is awesome", "I recommend watching this film to everyone, enjoy it"]


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

text = ', '.join(t for t in comments)
#print(text)
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(text)

plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
### WordCloud Visualization



text = ', '.join(t for t in movie['Review'])
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(text)

plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

In [ ]:
#https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/
sid_obj = SentimentIntensityAnalyzer()
sentiment_dict = sid_obj.polarity_scores("It tries to engage the people who are watching it. It tries to suprise people, however, this film is not interesting")
#sentiment_dict = sid_obj.polarity_scores("I love this film. It is wonderful!")
#sentiment_dict = sid_obj.polarity_scores("I am not sure about this film, no idea!")
print("Overall sentiment dictionary is : ", sentiment_dict)
print("Sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
print("Sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
print("Sentence was rated as ", sentiment_dict['pos']*100, "% Positive")

print("Sentence Overall Rated As", end=" ")

    # Decide sentiment as positive, negative, or neutral
if sentiment_dict['compound'] >= 0.05 :
    print("Positive")
elif sentiment_dict['compound'] <= -0.05 :
    print("Negative")
else :
    print("Neutral")

In [ ]:
#use this when you have an offline dataset
#movie =  pd.read_csv("F:\A-universities\WLV\workshop\Workshop3_IMDB_Dataset.csv")
#print(movie.head())
#print(movie.info())

In [ ]:
# Convert Rating to string first, then replace non-numeric values safely
movie['Rating'] = pd.to_numeric(movie['Rating'], errors='coerce')
#NaN represents missing or undefined data
# Fill NaN values with a default value (e.g., 0) before converting to integer
movie['Rating'] = movie['Rating'].fillna(0).astype(int)

### Extract Reviews and Ratings
## Text Processing and Analysis
### Importing Required Libraries




In [ ]:
# content = []
# for url in urls:
#     page = requests.get(url, timeout=2.50)
#     soup = BeautifulSoup(page.content, 'html.parser')
#     content.append(soup.find_all('div', class_='review-container'))

# Initialize lists to store extracted reviews and ratings
# Create DataFrame

import string
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split


##Sentiment Identification using VADER

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sentiments = []

for review in movie['Review']:
    score = sid.polarity_scores(review)['compound']
    if score >= 0.05:
        sentiments.append('positive')
    elif score <= -0.05:
        sentiments.append('negative')
    else:
        sentiments.append('neutral')

movie['Sentiment'] = sentiments

## Sentiment Classification using Machine Learning

movie['class-label'] = movie['Rating'].astype(int).apply(lambda x: '1' if x > 5 else ('-1' if x < 5 else '0'))
movie = movie[movie['class-label'] != '0']

#movie.to_csv('f:\myfile1.csv', index=False)
#analyzer is used to set the level of processing, it can be a character or a word level, 'word' or 'char'
#stop_words can provide you with a list of words that have to be removed from the data before calculations;
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
#Once you've created a vectorizer instance, it's time to obtain a TF-IDF matrix.
#You can use the fit_transform() class method and shape to print out its dimension:
X = tfidf_vectorizer.fit_transform(movie['Review'])
#print(f"Matrix dimension: {tfidf_matrix.shape}")
print(f"Matrix dimension: {X.shape}")
y = movie['class-label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

## Train and Evaluate SVM Classifier



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Not Approved", "Approved"])
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

#print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#How to interprete these metrics?
#https://www.statology.org/sklearn-classification-report/

In [ ]:
## Train and Evaluate Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Not Approved", "Approved"])
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix")
plt.show()

#print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#https://www.kaggle.com/datasets/columbine/imdb-dataset-sentiment-analysis-in-csv-format
#use this when you have an offline dataset
# movie1 =  pd.read_csv("F:\A-universities\WLV\workshop\Week 3\IMDB\Valid.csv")

if 'google.colab' in str(get_ipython()):
    from google.colab import drive, data_table
    drive.mount('/content/drive', force_remount=True)
    file_path = '/content/drive/My Drive/Colab Notebooks/data-mining/data/Valid.csv'
    data_table.enable_dataframe_formatter()
else:
    file_path = './data/Valid.csv'  # Local path

movie1 = pd.read_csv(file_path)

print(movie1.head())
print(movie.info())

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

text = ', '.join(t for t in movie1['text'])
stopwords = set(STOPWORDS)
wordcloud = WordCloud(stopwords=stopwords, background_color='white').generate(text)

plt.figure(figsize=(10,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import string
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split


##Sentiment Identification using VADER

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
sentiments = []

for review in movie1['text']:
    score = sid.polarity_scores(review)['compound']
    if score >= 0.05:
        sentiments.append('positive')
    elif score <= -0.05:
        sentiments.append('negative')
    else:
        sentiments.append('neutral')

movie1['Sentiment'] = sentiments

## Sentiment Classification using Machine Learning


# movie1.to_csv('f:\myfile1.csv', index=False)
#analyzer is used to set the level of processing, it can be a character or a word level, 'word' or 'char'
#stop_words can provide you with a list of words that have to be removed from the data before calculations;
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
#Once you've created a vectorizer instance, it's time to obtain a TF-IDF matrix.
#You can use the fit_transform() class method and shape to print out its dimension:
X = tfidf_vectorizer.fit_transform(movie1['text'])
#print(f"Matrix dimension: {tfidf_matrix.shape}")
print(f"Matrix dimension: {X.shape}")
y = movie1['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

## Train and Evaluate SVM Classifier



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

svc = SVC(kernel='sigmoid', gamma=1.0)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))

# Confusion Matrix
#conf_matrix = confusion_matrix(y_test, y_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Not Approved", "Approved"])
#disp.plot(cmap='Blues', values_format='d')
#plt.title("Confusion Matrix")
#plt.show()

#print('Confusion Matrix:', confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))

#How to interprete these metrics?
#https://www.statology.org/sklearn-classification-report/

In [ ]:

movie1['class-label'] = movie1['Sentiment'].apply(lambda x: 1 if x == "positive" else  0)
count=0
for i in range(0,len(set_2)-1):
    if set_1[i]==set_2[i]:
        count=count+1
print(count/len(set_2))